# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine

import json
import plotly
import plotly.express as px
import pandas as pd

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from flask import Flask
from flask import render_template, request

import joblib
from sqlalchemy import create_engine

pd.set_option('display.max_rows', 500)

In [ ]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('message', engine)
X = df.message.values
Y = df.iloc[:, 4:].values

We divide the message genres per categories

In [ ]:
# We take data from the fourth column. This column is 'genre', we have to do a groupby and sum
df_categories_genre = df.iloc[:, 4:].groupby('genre').sum().T

# We transpose the df
df_categories_genre = df_categories_genre.T

fig = go.Figure(data=[
    go.Bar(df_categories_genre, x=df_categories_genre, y=['direct', 'news', 'social'])])

# Change the bar mode
fig.update_layout(barmode='stack')

# Show the plot
fig.show()

We divide the number of message per genres, without categories

In [ ]:
# We take the column 'genre' and we do a groupby.
df_messages_genre = df.groupby('genre').count()

fig = px.bar(df_messages_genre, x=df_messages_genre.index, y='index', labels = {'x':'Message Genre', 'index' : '# Messages'})

# Show the plot
fig.show()

We divide the message per categories

In [ ]:
# We take data from the fith column. This column is 'related'
df_categories = pd.DataFrame(df.iloc[:, 5:].sum())

fig = px.bar(df_categories, y='count', x=df_categories, text_auto='.2s',
            title="Messages per categories")
            labels = {'index':' Categories', 'count' : '# Messages'}
    
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)

# Show the plot
fig.show()

Now we can see the same data of the last plot, but we plot the number of categories per message, instead of message per categories

In [ ]:
df_categories_message = pd.DataFrame()

# Create a column that sum the values from the fourth column. This sum will be the x axis
df_categories_message['# categories'] = df.iloc[:, 4:].sum(axis=1)

fig = px.bar(df_categories_message, x='# categories')

# Show the plot
fig.show()

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lower-case and lemmatize
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for t in tokens:
        clean_tok = lemmatizer.lemmatize(t).lower().strip()
        clean_tokens.append(clean_tok)
    
    # Word that are not in the stopword that we download from NLTK. We remove it 
    if remove_stopwords:
        clean_tokens = [word for word in clean_tokens if word not in stop_words] 
        
    return clean_tokens  
    #pass

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = [('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', MultiOutputClassifier(RandomForestClassifier()))]

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [ ]:
# Train pipeline
pipeline.fit(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
Y_pred = pipeline.predict(X_test)

In [ ]:
# We define the labels. From the fifth column, that is 'related'
label = df.iloc[:, 5:].columns.values

# The target names display names matching the labels in the same order. We return output as dict
report = classification_report(Y_test, Y_pred, target_names=label, output_dict = True)

# Show the result
#display(pd.DataFrame(report))

We need to do a comparison dataframe that we use and will use in the next step of the model

In [ ]:
comparison_df = (pd.DataFrame(report))['weighted avg']
comparison_df = pd.DataFrame(comparison_df).T.rename({'weighted avg':'RandomForestClassifier (default)'})
comparison_df

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# We define the parameters
param_grid = [
    {   "estimator_min_samples_leaf" : [1, 3, 12],
        "estimator_min_samples_split" : [3, 6],
        "estimator_max_depth": [None, 3, 12],
        "estimator_max_features" : [None, 'sqrt','log2']}
    ]


cv = GridSearchCV(default_pipeline, param_grid, cv =5, scoring=['f1_weight', 'precision_weight', 'recall_weight'],verbose=0, refit='f1_weighted', return_train_score=True, 
                  n_jobs=-1)

x = X_train
y = Y_train

cv.fit(x , y)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# Predict on the data
Y_pred = cv.predict(X_test)

In [ ]:
# We define the labels. From the fifth column, that is 'related'
label = df.iloc[:, 5:].columns.values

# The target names display names matching the labels in the same order. We return output as dict
cv_report = classification_report(Y_test, Y_pred, target_names=label, output_dict = True)

# Show the result
#display(pd.DataFrame(cv_report))

In [ ]:
# We show the result

param_name = {
    'estimator__min_samples_leaf_param': 'min_samples_leaf',
    'estimator__min_samples_split_param': 'min_samples_split',
    'estimator__max_depth_param' : 'max_depth', 
    'estimator__max_features_param': 'max_features',
    'test_f1_weight' : 'test_f1_weight',
    'test_precision_weight' : 'test_precision_weight',
    'test_recall_weight' : 'test_recall_weight'
    }

result = pd.DataFrame.from_dict(cv.result)

result_df = result[param_name.keys()]

result_df = result_df.rename(columns = param_name).sort_values('test_f1_weight')

result_df

In [ ]:
# We have to save the best parameters
cv_best_parameters = cv.best_params_
cv_best_parameters

In [ ]:
# We add the resultes row to the comparison df
row_cv = pd.DataFrame((pd.DataFrame(cv_report))['weighted avg']).T
comparison_df = pd.concat([comparison_df, row_cv]).rename({'weighted avg':'RandomForestClassifier (GridSearchCV)'})
comparison_df

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
One iof the mmost commom use in the machine learning is the

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.